In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd 
import os

In [36]:
url = "https://www.eba.europa.eu/all-news-and-press-releases?page="
counter = 1
track = True
href_list = list()
date_list = list()
title_list = list()
base = "https://www.eba.europa.eu"

# Send a GET request to the website
while track:
    # link to page number
    track = False
    concat = url + str(counter)
    response = requests.get(concat)

    # Create a BeautifulSoup object to parse the HTML content
    soup = BeautifulSoup(response.content, "html.parser")

    # Find all <a> elements with class="list-group-item"
    link_elements = soup.find_all("a", class_="list-group-item")

    # Find all <div> elements with class="list-group-item SecondaryInfo"
    info_elements = soup.find_all("p", class_="list-group-item-text SecondaryInfo")
    
    title_elements = soup.find_all("h3", class_ ="list-group-item-heading")

    current_datetime = datetime.now()
    current_month = current_datetime.strftime('%B')
    current_year = str(current_datetime.year)


    # Iterate over the extracted elements
    for link, info,title in zip(link_elements, info_elements,title_elements):
        href = link.get("href")
        text = info.text.strip()
        d = text[:10]
        date_object = datetime.strptime(d, "%d/%m/%Y")
        formatted_date = date_object.strftime("%d %B %Y")
        if current_year in formatted_date and current_month in formatted_date:
            track = True
            href_list.append(base+href)
            date_list.append(formatted_date)
            title_list.append(title.text.strip())
    counter += 1

df = pd.DataFrame({"Organisation" : ["European Banking Authority" for i in href_list],
                   "Title" : title_list,
                 "Link" : href_list,
                 "Date": date_list})

current = os.getcwd().replace("\\", "/")
file_path = "/Downloads/"
# Save to Downloads Folder
save_string = "EBA_updates.xlsx"
save_string = current + file_path + save_string # Comment out to save to directory file is in
df.to_excel(save_string, index = False)